In [1]:
import sys
sys.path.append('C:/Users/Jarvis/Desktop/train/')

In [2]:
%cd C:/Users/Jarvis/Desktop/

C:\Users\Jarvis\Desktop


In [3]:
%pwd

'C:\\Users\\Jarvis\\Desktop'

In [4]:
# we will import all required libraries for this tutorial in advance

import imgaug as ia
ia.seed(1)
# imgaug uses matplotlib backend for displaying images
%matplotlib inline
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 
# imageio library will be used for image input/output
import imageio
import pandas as pd
import numpy as np
import re
import os
import glob
# this library is needed to read XML files for converting it into CSV
import xml.etree.ElementTree as ET
import shutil

In [5]:
# Function that will extract column data for our CSV file
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [6]:
xml_to_csv('C:/Users/Jarvis/Desktop/train/')

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Acheiropoietos230.jpg,300,300,Acheiropoietos,1,40,299,239
1,Acheiropoietos231.jpg,300,300,Acheiropoietos,106,37,255,201
2,Acheiropoietos232.jpg,300,300,Acheiropoietos,5,87,290,234
3,Acheiropoietos234.jpg,300,300,Acheiropoietos,51,56,248,244
4,Acheiropoietos235.jpg,300,300,Acheiropoietos,98,50,300,249
...,...,...,...,...,...,...,...,...
3770,White_Tower_95.jpg,300,300,White Tower,131,59,202,196
3771,White_Tower_96.jpg,300,300,White Tower,106,53,207,225
3772,White_Tower_97.jpg,300,300,White Tower,24,63,227,178
3773,White_Tower_98.jpg,300,300,White Tower,96,43,245,260


In [7]:
# apply xml_to_csv() function to convert all XML files in images/ folder into labels.csv
labels_df = xml_to_csv('C:/Users/Jarvis/Desktop/train/')
labels_df.to_csv(('labels.csv'), index=None)
print('Successfully converted xml to csv.')

Successfully converted xml to csv.


In [37]:
labels_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Acheiropoietos230.jpg,300,300,Acheiropoietos,1,40,299,239
1,Acheiropoietos231.jpg,300,300,Acheiropoietos,106,37,255,201
2,Acheiropoietos232.jpg,300,300,Acheiropoietos,5,87,290,234
3,Acheiropoietos234.jpg,300,300,Acheiropoietos,51,56,248,244
4,Acheiropoietos235.jpg,300,300,Acheiropoietos,98,50,300,249
...,...,...,...,...,...,...,...,...
3770,White_Tower_95.jpg,300,300,White Tower,131,59,202,196
3771,White_Tower_96.jpg,300,300,White Tower,106,53,207,225
3772,White_Tower_97.jpg,300,300,White Tower,24,63,227,178
3773,White_Tower_98.jpg,300,300,White Tower,96,43,245,260


In [22]:
# This setup of augmentation parameters will pick two of four given augmenters and apply them in random order
aug = iaa.SomeOf(1, [    
    iaa.Affine(scale=(0.5, 1.0)),
    #iaa.Affine(shear =(-30, 30)),
    iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
    iaa.Multiply((0.5, 1.5)),
    iaa.GaussianBlur(sigma=(0.75, 1.0)),
    iaa.AdditiveGaussianNoise(scale=(0.03*255, 0.04*255)),
    iaa.LinearContrast((0.75, 1.5))
])           

In [23]:
# function to convert BoundingBoxesOnImage object into DataFrame
def bbs_obj_to_df(bbs_object):
#     convert BoundingBoxesOnImage object into array
    bbs_array = bbs_object.to_xyxy_array()
#     convert array into a DataFrame ['xmin', 'ymin', 'xmax', 'ymax'] columns
    df_bbs = pd.DataFrame(bbs_array, columns=['xmin', 'ymin', 'xmax', 'ymax'])
    return df_bbs

In [24]:
def image_aug(df, images_path, aug_images_path, image_prefix, augmentor):
    # create data frame which we're going to populate with augmented image info
    aug_bbs_xy = pd.DataFrame(columns=
                              ['filename','width','height','class', 'xmin', 'ymin', 'xmax', 'ymax']
                             )
    grouped = df.groupby('filename')
    
    for filename in df['filename'].unique():
    #   get separate data frame grouped by file name
        group_df = grouped.get_group(filename)
        group_df = group_df.reset_index()
        group_df = group_df.drop(['index'], axis=1)   
    #   read the image
        image = imageio.imread(images_path+filename)
    #   get bounding boxes coordinates and write into array        
        bb_array = group_df.drop(['filename', 'width', 'height', 'class'], axis=1).values
    #   pass the array of bounding boxes coordinates to the imgaug library
        bbs = BoundingBoxesOnImage.from_xyxy_array(bb_array, shape=image.shape)
    #   apply augmentation on image and on the bounding boxes
        image_aug, bbs_aug = augmentor(image=image, bounding_boxes=bbs)
    #   disregard bounding boxes which have fallen out of image pane    
        bbs_aug = bbs_aug.remove_out_of_image()
    #   clip bounding boxes which are partially outside of image pane
        bbs_aug = bbs_aug.clip_out_of_image()
        
    #   don't perform any actions with the image if there are no bounding boxes left in it    
        if re.findall('Image...', str(bbs_aug)) == ['Image([]']:
            pass
        
    #   otherwise continue
        else:
        #   write augmented image to a file
            imageio.imwrite(aug_images_path+image_prefix+filename, image_aug)  
        #   create a data frame with augmented values of image width and height
            info_df = group_df.drop(['xmin', 'ymin', 'xmax', 'ymax'], axis=1)    
            for index, _ in info_df.iterrows():
                info_df.at[index, 'width'] = image_aug.shape[1]
                info_df.at[index, 'height'] = image_aug.shape[0]
        #   rename filenames by adding the predifined prefix
            info_df['filename'] = info_df['filename'].apply(lambda x: image_prefix+x)
        #   create a data frame with augmented bounding boxes coordinates using the function we created earlier
            bbs_df = bbs_obj_to_df(bbs_aug)
        #   concat all new augmented info into new data frame
            aug_df = pd.concat([info_df, bbs_df], axis=1)
        #   append rows to aug_bbs_xy data frame
            aug_bbs_xy = pd.concat([aug_bbs_xy, aug_df])            
    
    # return dataframe with updated images and bounding boxes annotations 
    aug_bbs_xy = aug_bbs_xy.reset_index()
    aug_bbs_xy = aug_bbs_xy.drop(['index'], axis=1)
    return aug_bbs_xy

In [25]:
augmented_images_df = image_aug(labels_df, 'train/', 'augmented/', 'Aug1_', aug)
augmented_images_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Aug1_Acheiropoietos230.jpg,300,300,Acheiropoietos,1.000000,40.000000,299.00000,239.000000
1,Aug1_Acheiropoietos231.jpg,300,300,Acheiropoietos,106.000000,37.000000,255.00000,201.000000
2,Aug1_Acheiropoietos232.jpg,300,300,Acheiropoietos,5.000000,87.000000,290.00000,234.000000
3,Aug1_Acheiropoietos234.jpg,300,300,Acheiropoietos,51.000000,56.000000,248.00000,244.000000
4,Aug1_Acheiropoietos235.jpg,300,300,Acheiropoietos,154.545792,25.238430,300.00000,224.238434
...,...,...,...,...,...,...,...,...
3770,Aug1_White_Tower_95.jpg,300,300,White Tower,168.527130,70.551292,239.52713,207.551300
3771,Aug1_White_Tower_96.jpg,300,300,White Tower,106.000000,53.000000,207.00000,225.000000
3772,Aug1_White_Tower_97.jpg,300,300,White Tower,24.000000,63.000000,227.00000,178.000000
3773,Aug1_White_Tower_98.jpg,300,300,White Tower,96.000000,43.000000,245.00000,260.000000


In [26]:
augmented_images_df2 = image_aug(labels_df, 'train/', 'augmented/', 'Aug2_', aug)
augmented_images_df2

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Aug2_Acheiropoietos230.jpg,300,300,Acheiropoietos,1.000000,40.000000,299.000000,239.000000
1,Aug2_Acheiropoietos231.jpg,300,300,Acheiropoietos,106.000000,37.000000,255.000000,201.000000
2,Aug2_Acheiropoietos232.jpg,300,300,Acheiropoietos,0.000000,101.244545,245.342514,248.244537
3,Aug2_Acheiropoietos234.jpg,300,300,Acheiropoietos,51.000000,56.000000,248.000000,244.000000
4,Aug2_Acheiropoietos235.jpg,300,300,Acheiropoietos,98.000000,50.000000,300.000000,249.000000
...,...,...,...,...,...,...,...,...
3770,Aug2_White_Tower_95.jpg,300,300,White Tower,131.000000,59.000000,202.000000,196.000000
3771,Aug2_White_Tower_96.jpg,300,300,White Tower,106.000000,53.000000,207.000000,225.000000
3772,Aug2_White_Tower_97.jpg,300,300,White Tower,24.000000,63.000000,227.000000,178.000000
3773,Aug2_White_Tower_98.jpg,300,300,White Tower,83.620346,90.147125,232.620346,300.000000


In [39]:
# Concat resized_images_df and augmented_images_df together and save in a new all_labels.csv file
all_labels_df = pd.concat([labels_df, augmented_images_df, augmented_images_df2])
all_labels_df.to_csv('train_aug_labels.csv', index=False)